In [1]:
#imports
import numpy as np
import cvxpy as cp
import matplotlib.pyplot as plt

Quadcopter parameters

In [2]:
#quadcopter parameters
g = 9.81
m = 1.
Ix = 8.1 * 1e-3
Iy = 8.1 * 1e-3
Iz = 14.2 * 1e-3

Non-linear dynamcis

In [3]:
#non-linear dynamics
def f(x, u):
    x1, x2, theta1, theta2, y1, y2, phi1, phi2, z1, z2, psi1, psi2 = x.reshape(-1).tolist()
    tau_y, tau_x, ft, tau_z = u.reshape(-1).tolist()
    dot_x = np.array([
     x2,
     ft/m*(np.sin(phi1)*np.sin(psi1)+np.cos(phi1)*np.cos(psi1)*np.sin(theta1)),
     theta2,
     (Iz-Ix)/Iy*phi2*psi2+tau_y/Iy,
     y2,
     ft/m*(np.cos(phi1)*np.sin(psi1)*np.sin(theta1)-np.cos(psi1)*np.sin(phi1)),
     phi2,
     (Iy-Iz)/Ix*theta2*psi2+tau_x/Ix,
     z2,
     -g + ft/m*np.cos(phi1)*np.cos(theta1),# 
     psi2,
     (Ix-Iy)/Iz*phi2*theta2+tau_z/Iz])
    return dot_x

Linear model

In [4]:
# X-subsystem
# The state variables are x, dot_x, pitch, dot_pitch
Ax = np.array(
    [[0.0, 1.0, 0.0, 0.0],
     [0.0, 0.0, g, 0.0],
     [0.0, 0.0, 0.0, 1.0],
     [0.0, 0.0, 0.0, 0.0]])
Bx = np.array(
    [[0.0],
     [0.0],
     [0.0],
     [1 / Ix]])

# Y-subsystem
# The state variables are y, dot_y, roll, dot_roll
Ay = np.array(
    [[0.0, 1.0, 0.0, 0.0],
     [0.0, 0.0, -g, 0.0],
     [0.0, 0.0, 0.0, 1.0],
     [0.0, 0.0, 0.0, 0.0]])
By = np.array(
    [[0.0],
     [0.0],
     [0.0],
     [1 / Iy]])

# Z-subsystem
# The state variables are z, dot_z
Az = np.array(
    [[0.0, 1.0],
     [0.0, 0.0]])
Bz = np.array(
    [[0.0],
     [1 / m]])

# Yaw-subsystem
# The state variables are yaw, dot_yaw
Ayaw = np.array(
    [[0.0, 1.0],
     [0.0, 0.0]])
Byaw = np.array(
    [[0.0],
     [1 / Iz]])

A1 = np.concatenate((Ax, np.zeros((4,8))), axis = 1)
A2 = np.concatenate((np.zeros((4,4)), Ay, np.zeros((4,4))),axis = 1)
A3 = np.concatenate((np.zeros((2,8)), Az, np.zeros((2,2))),axis = 1)
A4 = np.concatenate((np.zeros((2,10)), Ayaw), axis = 1)

B1 = np.concatenate((Bx, np.zeros((4,3))), axis = 1)
B2 = np.concatenate((np.zeros((4,1)), By, np.zeros((4,2))),axis = 1)
B3 = np.concatenate((np.zeros((2,2)), Bz, np.zeros((2,1))),axis = 1)
B4 = np.concatenate((np.zeros((2,3)), Byaw), axis = 1)

A = np.concatenate((A1,A2,A3,A4))
B = np.concatenate((B1,B2,B3,B4))

MRAC

In [5]:
def mrac(Aold,Bold,x0,x1,u,dt):
    try:

        A = cp.Variable((12,12))
        B = cp.Variable((12,4))

        cost = cp.norm(A - Aold, "fro") + cp.norm(B - Bold, "fro")
        constraints = [x1 == x0 + (A@x0 + B@u + B@[0,0,-9.81,0])*dt]

        prob = cp.Problem(cp.Minimize(cost), constraints)

        prob.solve()

        print(prob.status)

        if prob.status == "infeasible":
            return False

        return A.value, B.value

    except:
        return False

MPC

In [6]:
def mpc(A,B,Q,R,P,x0,x_traj,track,N,posX,posN,dt,N0):

    if track == 0:
        x_opt = cp.Variable((N+1,12))
        u_opt = cp.Variable((N,4))
    else: 
        x_opt = cp.Variable((track*N+1,12))
        u_opt = cp.Variable((track*N,4))

    termX = posX + track + 1

    if posN == 0:
        termX -= 1

    if termX >= len(x_traj):
        termX = len(x_traj) - 1

    cost = cp.quad_form(x_opt[-1] - x_traj[termX], P)
    constraints = [x0 == x_opt[0]] #[x0[0] == x_opt[0][0], x0[4] == x_opt[0][4], x0[8] == x_opt[0][8]]
    
    if posN != 0:
        constraints += [x_traj[posX+1][0] == x_opt[N0-posN][0], x_traj[posX+1][4] == x_opt[N0-posN][4], x_traj[posX+1][8] == x_opt[N0-posN][8]]
    
    for i in range(track-1):
        if posX < len(x_traj) - track - i:
            constraints += [x_traj[posX+2 + i][0] == x_opt[(i+1)*N + N-posN][0], x_traj[posX+2 + i][4] == x_opt[(i+1)*N + N-posN][4], x_traj[posX+2 + i][8] == x_opt[(i+1)*N + N-posN][8]]
  
    for i in range(track*N):
        cost += cp.quad_form(x_opt[i] - x_traj[termX],Q) + cp.quad_form(u_opt[i],R)
        constraints += [x_opt[i+1] == x_opt[i] + dt*(A@x_opt[i] + B@u_opt[i] + B@[0,0,-9.81,0])]

    if track == 0:
        for i in range(N):
            cost += cp.quad_form(x_opt[i] - x_traj[termX],Q) + cp.quad_form(u_opt[i],R)
            constraints += [x_opt[i+1] == x_opt[i] + dt*(A@x_opt[i] + B@u_opt[i] + B@[0,0,-9.81,0])]

    prob = cp.Problem(cp.Minimize(cost), constraints)

    prob.solve()

    #print(prob.status)

    if prob.status == "infeasible":
        return False

    return x_opt.value, u_opt.value


MPC Parameters

In [ ]:
#MPC parameters 1
q = 1
r = 1
p = 1
Q = q*np.identity(12)
R = p*np.identity(4)
P = r*np.identity(12)

N = 25
N0 = N
dt = 0.01
track = 3

x_traj = np.array([
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0],\
                  [2.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  ])

x_traj = x_traj/100

In [ ]:
#MPC parameters 2
q = 1
r = 1
p = 1
Q = q*np.identity(12)
R = p*np.identity(4)
P = r*np.identity(12)

N = 25
N0 = N
dt = 0.01
track = 3

x_traj = np.array([
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [2.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [3.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [2.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  ])

x_traj = x_traj/100

In [7]:
#MPC parameters 3
q = 1
r = 1
p = 1
Q = q*np.identity(12)
R = p*np.identity(4)
P = r*np.identity(12)

N = 25
N0 = N
dt = 0.01
track = 3

x_traj = np.array([
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [0.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 4.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 4.0,0.0,0.0,0.0],\
                  ])

x_traj = x_traj/100

In [7]:
#MPC parameters 4
q = 1
r = 1
p = 1
Q = q*np.identity(12)
R = p*np.identity(4)
P = r*np.identity(12)

N = 25
N0 = N
dt = 0.01
track = 3

x_traj = np.array([
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 4.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  ])

x_traj = x_traj/100

In [7]:
#MPC parameters 5
q = 1
r = 1
p = 1
Q = q*np.identity(12)
R = p*np.identity(4)
P = r*np.identity(12)

N = 25
N0 = N
dt = 0.01
track = 3

x_traj = np.array([
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 2.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 3.0,0.0,0.0,0.0],\
                  [1.0,0.0,0.0,0.0, 1.0,0.0,0.0,0.0, 3.0,0.0,0.0,0.0],\
                  [0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 3.0,0.0,0.0,0.0],\
                  ])

x_traj = x_traj/100

Simulation

In [ ]:
#simulate

plotX = []
plotY = []
plotZ = []
flag = 0
X = np.array([0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0, 0.0,0.0,0.0,0.0])
print(X)
for i in range(len(x_traj)-1):
    if i + track == len(x_traj) - 1:
        track -= 1

    for j in range(N):

        if mpc(A,B,Q,R,P,X,x_traj,track,N,i,j,dt,N0) == False: 

            Xprev = X
            X = X + f(X,np.zeros(4))*dt
            print("non")

            if mrac(A,B,Xprev,X,np.zeros(4),dt) == False: 
                pass

            else: 
                A,B = mrac(A,B,Xprev,X,np.zeros(4),dt)
        
        else:
            x_mpc, u_mpc = mpc(A,B,Q,R,P,X,x_traj,track,N,i,j,dt,N0)
        
            X = X + f(X,u_mpc[0])*dt

            print("done",i,j)

        #X = np.array(x_mpc[1])

            if mrac(A,B,x_mpc[0],X,u_mpc[0],dt) == False: 
                pass
            else: 
                A,B = mrac(A,B,x_mpc[0],X,u_mpc[0],dt)
        

        plotX.append(X[0]*100)
        plotY.append(X[4]*100)
        plotZ.append(X[8]*100)

        if track == 0:
            N = N-1


Outputs

In [ ]:
#plot 1
trackX = []
trackY = []
trackZ = []
for i in x_traj:
    trackX.append(i[0]*100)
    trackY.append(i[4]*100)
    trackZ.append(i[8]*100) 

track = []
for j in range(len(x_traj)):
    track.append(j*N0)   


plt.subplot(311)
plt.plot(plotX, color = "blue")
plt.plot(track, trackX, "o", color = "blue")
plt.legend(["X"])
plt.subplot(312)
plt.plot(plotY, color = "orange")
plt.plot(track, trackY, "o", color = "orange")
plt.legend(["Y"])
plt.subplot(313)
plt.plot(plotZ, color = "green")
plt.plot(track, trackZ, "o", color = "green")
plt.legend(["Z"])

In [ ]:
#plot 2
ax = plt.figure().add_subplot(projection='3d')
ax.plot(plotX, plotY, plotZ)
ax.plot(trackX, trackY, trackZ,"*")

In [ ]:
print(trackX, trackY, trackZ)